In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import os
import matplotlib.pyplot as plt
import logging as log
import src.config as config
import sys
import src.utilities as ut
from src.update_databases import tinkerforge_update

%matplotlib inline
%load_ext autoreload
%autoreload 2

Tinkerforge up-to-date
EnergyMeter up-to-date


In [3]:
logpath = './logs'
onlyfiles = [os.path.join(logpath, f) for f in os.listdir(logpath) if os.path.isfile(os.path.join(logpath, f))]
onlyfiles.sort(key=lambda x: os.path.getmtime(x))
onlyfiles.reverse()
tf_skip = []
em_skip = False
for logfile in onlyfiles:
    with open(logfile,encoding='latin-1') as f:
        f = f.read().splitlines()
        f.reverse()
        data = []
        for line in f:
            line = line.split(' -- ')
            if 'TinkerForge Datenbanken exportiert!' in line[2]:
                timecode = line[0].strip()
                bui = line[2].split(':')[0].strip()
                info = line[2].split('|')
                start = dt.datetime.strptime(info[1].strip(), "%Y-%m-%d %H:%M:%S")
                end = dt.datetime.strptime(info[2].strip(), "%Y-%m-%d %H:%M:%S")
                if end.date() < (dt.date.today() - dt.timedelta(1)):
                    tf_skip.append(bui)
            elif 'Moline Datenbanken exportiert!' in line[2]:
                timecode = line[0].strip()
                bui = line[2].split(':')[0].strip()
                info = line[2].split('|')
                start = dt.datetime.strptime(info[1].strip(), "%Y-%m-%d %H:%M:%S")
                end = dt.datetime.strptime(info[2].strip(), "%Y-%m-%d %H:%M:%S")
                if end.date() < (dt.date.today() - dt.timedelta(1)):
                    em_skip = True
if tf_skip != buid:
    tinkerforge_update(skip=tf_skip)
if not em_skip:
    moline_update()


21.02.2022 22:22:39 -- INFO -- ------ Starte TinkerForge Update------
21.02.2022 22:22:39 -- INFO -- LB wurde übersprungen.
21.02.2022 22:22:39 -- INFO -- MH: Suche vorhandene Datenbank.


Tinkerforge up-to-date
EnergyMeter up-to-date


21.02.2022 22:22:49 -- INFO -- MH: Datenbank geöffnet, letzter Eintrag vom 2022-02-21!
21.02.2022 22:22:50 -- INFO -- MH: Durchsuche Dropbox nach neuen Datensätzen.


21.02.2022 22:22:55 -- INFO -- MW: Suche vorhandene Datenbank.
21.02.2022 22:23:05 -- INFO -- MW: Datenbank geöffnet, letzter Eintrag vom 2022-02-18!
21.02.2022 22:23:05 -- INFO -- Ziegelhaus liefert seit 17.02.2022 (4 Tage(n)) keine neuen Daten mehr.
21.02.2022 22:23:05 -- INFO -- MW: Durchsuche Dropbox nach neuen Datensätzen.


21.02.2022 22:23:09 -- INFO -- WD: Suche vorhandene Datenbank.
21.02.2022 22:23:10 -- INFO -- WD: Datenbank geöffnet, letzter Eintrag vom 2022-02-21!
21.02.2022 22:23:11 -- INFO -- WD: Durchsuche Dropbox nach neuen Datensätzen.


21.02.2022 22:23:15 -- INFO -- PM: Suche vorhandene Datenbank.
21.02.2022 22:23:16 -- INFO -- PM: Datenbank geöffnet, letzter Eintrag vom 2022-02-21!
21.02.2022 22:23:16 -- INFO -- PM: Durchsuche Dropbox nach neuen Datensätzen.


21.02.2022 22:23:22 -- INFO -- Offline Benachrichtigung gesendet!
21.02.2022 22:23:22 -- INFO -- ------TinkerForge Update beendet!------


Message sent!
